In [1]:
pip install pydicom

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nibabel

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-python

In [1]:
import pydicom
import nibabel as nib
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2

In [2]:
from pydicom.data import get_testdata_file
from pydicom import dcmread


In [2]:
from google.colab import files

files
files.upload()


Saving cell.jpg to cell.jpg


{'cell.jpg': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xfe\x00>CREATOR: gd-jpeg v1.0 (using IJG JPEG v62), default quality\n\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x02\xd0\x03\xc0\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\x

In [4]:
archivos_procesados = {}

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os


In [3]:
#los dos diccionarios enn los que almacena la informacion
pacientes_creados = {}
archivos_procesados ={}


class Paciente:
    def __init__(self, edad, id, imagen_asociada):
        self.edad = edad
        self.id = id
        self.imagen_asociada = imagen_asociada

def cargar_pacientes(ruta):
    for file in os.listdir(ruta):
        if file.endswith('.dcm'):
            ruta_dicom = os.path.join(ruta, file)
            leer_dicom = pydicom.dcmread(ruta_dicom)

            edad = leer_dicom.PatientBirthDate
            id = leer_dicom.PatientID
            imagen_dicom = leer_dicom.pixel_array

            nifti_img = nib.Nifti1Image(imagen_dicom, affine=np.eye(4))
            nifti_ruta = os.path.join(ruta, f"{id}.nii")
            nib.save(nifti_img, nifti_ruta)

            paciente = Paciente(edad, id, nifti_ruta)#crea un objeto en la clase pacientes)
            pacientes_creados[id]= paciente
            archivos_procesados[id]= ruta_dicom
    print("pacientes cargados exitosamente")
    print(pacientes_creados)
    print(archivos_procesados)

def rotacion(clave, angulo):
    if clave in archivos_procesados:
        leer_dicom = pydicom.dcmread(archivos_procesados[clave])
        imagen = leer_dicom.pixel_array

        # Mostrar imágenes
        plt.subplot(1, 2, 1)
        plt.title('Imagen Original')
        plt.imshow(imagen)
        row, col = np.shape(imagen)


        plt.subplot(1, 2, 2)
        MR = cv2.getRotationMatrix2D((col/2,row/2), angulo ,1)
        #Rotación
        rot = cv2.warpAffine(imagen,MR,(col,row))
        plt.imshow(rot)
        plt.title('Imagen Rotada')

        plt.show()

        cv2.imwrite(f"{clave}_rotada.png", rot)
        archivos_procesados[f"{clave}_rotada.png"] = f"{clave}_rotada.png"


def mostrar_imagen():
    ruta_imagen = input("Ingrese la ruta del archivo JPG o PNG: ")
    if os.path.exists(ruta_imagen):

        # Para guardar las imagenes y que se diferencien por sus rutas y formatos(.jpg o .png)
        nombre_original = os.path.basename(ruta_imagen)
        nombre_sin_extension, extension = os.path.splitext(nombre_original)

        # Leer la imagen
        img0 = cv2.imread(ruta_imagen)
        img = cv2.cvtColor(img0, cv2.COLOR_BGR2GRAY)
        print("Imagen cargada exitosamente.")

        # Pide el tamaño del kernel y el umbral
        tamano_kernel = int(input("Ingrese el tamaño del kernel: "))
        Umb = int(input("Ingrese el umbral(0-255): "))

        # Umbral binary
        Umb,imgB=cv2.threshold(img,Umb,255,cv2.THRESH_BINARY)

        # Kernel para la tranformación
        kernel = np.ones((tamano_kernel,tamano_kernel),np.uint8)
        imaDil = cv2.dilate(imgB, kernel, iterations=1)

        imagen_combinada = cv2.addWeighted(imgB, 0.5, imaDil, 0.5, 0)


        # Imagen binarizada con umbral
        plt.figure(figsize=(8,8))
        plt.imshow(imagen_combinada, cmap='gray', vmin=0, vmax=255)
        plt.title(f'Imagen binarizada\nUmbral: {Umb}, Tamaño del kernel: {tamano_kernel}')
        plt.axis('off')
        plt.show()

        # Guardar la imagen con sus nombres específicos
        nombre_imagen_binarizada = f"{nombre_sin_extension}_binarizada{extension}"
        cv2.imwrite(nombre_imagen_binarizada, imagen_combinada)

        # para guardarla en el diccionario archivos_procesos
        archivos_procesados[nombre_imagen_binarizada] = nombre_imagen_binarizada

        print(f"Imagen guardada como '{nombre_imagen_binarizada}'.")
        print(archivos_procesados)

    else:
        print("La ruta ingresada no es válida o el archivo no existe.")



def menu_principal():
    while True:
        menu = input('''Menu
        1. Ingresar paciente
        2. Ingresar imagen (jpg o png)
        3. Rotar imagen
        4. Binarizar y transformar morfologicamente la imagen
        5. Salir
        ''')
        if menu =="1":
            #profe nosotras hicimos las pruebas con los archivos de DICOM_T3 asi que si que se prueba con otra ruta puedes que no lea ya que
            #este archivo tenia era PatientBirthDate y no PatientAge
            opcion = input("ingrese la ruta de archivo")
            #"DICOM_T3/DICOM_T3/T1" esta es la ruta que utilizamos
            cargar_pacientes(opcion)


        elif menu =="2":
            ruta_imagen = input("Ingrese la ruta del archivo jpg o png: ")
            if os.path.exists(ruta_imagen):
                nombre_archivo = os.path.basename(ruta_imagen)
                archivos_procesados[nombre_archivo] = ruta_imagen
                print(f"Imagen {nombre_archivo} almacenada exitosamente.")
            else:
                print("La ruta ingresada no es válida o el archivo no existe.")


        elif menu == "3":
            clave = input("ingrese la id del paciente")
            angulo = int(input("ingrese el angulo que desea rotar (90, 180 o 270)"))
            if angulo == 90:
                rotacion(clave, angulo)
            elif angulo == 180:
                rotacion(clave, angulo)
            elif angulo == 270:
                rotacion(clave, angulo)
            else:
                print("ingrese una opcion valida")


        elif menu == "4":
            mostrar_imagen()


        elif menu == "5":
            print("Saliendo del programa")
            break
        else:
            print("opcion no valida")


if __name__ == "__main__":
  menu_principal()



Menu
        1. Ingresar paciente
        2. Ingresar imagen (jpg o png)
        3. Rotar imagen
        4. Binarizar y transformar morfologicamente la imagen
        5. Salir
         5


Saliendo del programa
